In [1]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
import math
import networkx as nx
import queue
import math

**Input**: Undirected graph $G = (V, E)$, threshold function $\theta$

**Input**: parameter values for $t_{lim}, n_{ind}, p_e, p_m, prob_{elite}, seed$

In [3]:
class BRKGA:
    def __init__(self, G, n, m, tlim=100, nind=46, pe=0.24, pm=0.13, pelite=0.69, seed=True):
        self.G = G
        self.n = n
        self.m = m
        self.tlim = tlim
        self.nind = nind
        self.pe = pe
        self.pm = pm
        self.pelite = pelite
        self.seed = seed
        self.deg = np.zeros(self.n)
        for i in range(self.n):
            self.deg[i] = len(self.G[i])

    def run(self, inplace = True):
        self.P = np.random.rand(self.nind, self.n)
        if self.seed == True:   self.P[0] = np.full(self.n, 0.5)
        start = timeit.default_timer()
        while timeit.default_timer() - start < self.tlim:
            elite_id = self.get_elite_id()
            non_elite_id = np.setdiff1d(np.arange(self.nind), elite_id)
            Pm = self.mutate()
            Pc = self.crossover(elite_id, non_elite_id, self.nind - Pm.shape[0] - elite_id.shape[0])
            self.P = np.concatenate((self.P[elite_id], Pm, Pc))
            print(np.min(self.score), end=' ')
        if not inplace:     return self.P

    def get_pe(self):      return self.pe
    def get_pm(self):       return self.pm
    def get_pelite(self):   return self.pelite

    def get_elite_id(self):
        pelite = self.get_pe()
        Pe_size = math.ceil(self.nind * pelite)
        self.eval()
        elite_id = np.argpartition(-self.score, -Pe_size)[-Pe_size:]
        return elite_id

    def mutate(self):
        pm = self.get_pm()
        Pm_size = math.ceil(self.nind * pm)
        Pm = np.random.rand(Pm_size, self.n)
        return Pm

    def eval(self):
        self.score = np.zeros(self.nind)
        for i in range(self.nind):
            self.score[i] = np.sum(self.MDG(self.P[i]))


    def crossover(self, elite_id, non_elite_id, Pc_size):
        # create Pc_size new individuals by crossover: choose 1 parent from elite_id and 1 from non_elite_id
        # for each gene, choose 1 from 2 parents with prob pelite and 1 - pelite
        Pc = np.zeros((Pc_size, self.n))
        pelite = self.get_pelite()
        for i in range(Pc_size):
            x = np.random.choice(elite_id)
            y = np.random.choice(non_elite_id)
            for j in range(self.n):
                if np.random.rand() < pelite:   Pc[i][j] = self.P[x][j]
                else:                           Pc[i][j] = self.P[y][j]
        return Pc

    def phi(self, Cov):
        d = np.zeros(self.n)
        res = np.zeros(self.n)
        q = queue.Queue()
        for i in range(self.n):
            if Cov[i] == 1:
                q.put(i)
        while not q.empty():
            u = q.get()
            if res[u] == 1:     continue
            res[u] = 1
            for v in self.G[u]:
                d[v] += 1
                if d[v] == (self.deg[v]+1) // 2:
                    q.put(v)
        return res

    def MDG(self, w=1):
        S = np.zeros(self.n)
        Cov = np.zeros(self.n)
        d = self.deg * w
        while np.sum(Cov) < self.n:
            id, maxdeg = -1, -1
            for i in range(self.n):
                if Cov[i] == 0:
                    if d[i] > maxdeg:
                        maxdeg = d[i]
                        id = i
            Cov[id] = 1
            Cov = self.phi(Cov)
            S[id] = 1
        return S

In [4]:
def sample_waiting_time(p):
    """
    Samples the number of independent tries until a given random event happens.
    'p' should be the probability of the event happening in one try.
    """
    x = np.random.rand()
    # We use log1p(z) to compute precisely log(1 + z) :
    return 1 + int(np.log1p(- x) // np.log1p(- p))

def sample(cdf):
    """
    Samples a random positive bounded integer according to the Cumulative Distribution Function
	given as input (under the form of an array 'cdf', with cdf[i + 1] >= cdf[i]
    and cdf[-1] = 1. Note that these verifications are not performed by the function).
    
    The resulting integer X has the property that
        Pr(X <= i + 1) = cdf[i], for i in [0, ..., n - 1] :
    
    Pr(X = 1) = cdf[0]
    Pr(X = 2) = cdf[1] - cdf[0]
    ...
    Pr(X = n) = cdf[n - 1] - cdf[n - 2] - ... - cdf[0]
    """
    n = len(cdf)
    x = np.random.random()
    if x < cdf[0]:
        return 1
    low = 0
    high = n - 1
    while high - low > 1:
        mid = (low + high) >> 1
        if x > cdf[mid]:
            low = mid
        else:
            high = mid
    return high + 1

class fastBRKGA(BRKGA):
    def __init__(self, G, n, m, tlim=100, nind=46, seed=True, beta=1.5):
        super().__init__(G, n, m, tlim, nind, seed)
        self.beta = beta
        self.pe_distribution = np.linspace(1, 15, 15) ** (-beta)
        prefix_sum = np.cumsum(self.pe_distribution)
        self.pe_power_law = prefix_sum / prefix_sum[-1]

        self.pm_distribution = np.linspace(1, 20, 20) ** (-beta)
        prefix_sum = np.cumsum(self.pm_distribution)
        self.pm_power_law = prefix_sum / prefix_sum[-1]

        self.pelite_distribution = np.linspace(1, 30, 30) ** (-beta)
        prefix_sum = np.cumsum(self.pelite_distribution)
        self.pelite_power_law = prefix_sum / prefix_sum[-1]

    def get_pe(self):
        # domain: [0.1, 0.25]
        x = sample(self.pe_power_law)
        return 0.1 + 0.01 * (15-x)

    def get_pm(self):
        # domain: [0.1, 0.3]
        x = sample(self.pm_power_law)
        return 0.1 + 0.01 * x

    def get_pelite(self):
        # domain: [0.51, 0.8] 
        x = sample(self.pelite_power_law)
        return 0.51 + 0.01 * x
        

In [5]:
#read file and create graph
def read_file(filename):
    inp = open(filename, 'r')
    x = inp.readlines()
    n, _, m = map(int, x[1].split())
    G = [[] for i in range(n)]
    for i in range(2, len(x)):
        u, v, _ = map(int, x[i].split())
        G[u-1].append(v-1)
        G[v-1].append(u-1)
    return G, n, m

G, n, m = read_file('jazz/jazz.mtx')
brkga = BRKGA(G, n, m, tlim=300)
S = brkga.MDG()
# print index where S[i] = 1
id = np.where(S == 1)[0]
print(len(id),id)

#brkga.run()
fast_brkga = fastBRKGA(G, n, m, tlim=300)
fast_brkga.run()

29 [  4   6  53  59  68  69  82  85  95  98  99 100 104 107 121 130 131 134
 135 148 152 157 163 166 167 169 173 191 193]
29.0 29.0 28.0 27.0 27.0 26.0 25.0 25.0 25.0 25.0 25.0 25.0 25.0 25.0 24.0 24.0 24.0 24.0 24.0 24.0 24.0 24.0 24.0 23.0 23.0 23.0 23.0 23.0 23.0 23.0 23.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.0 22.

Traceback (most recent call last):
  File "/Users/vuhoangnguyen/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_64332/2012243271.py", line 22, in <module>
    fast_brkga.run()
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_64332/2842694499.py", line 21, in run
    elite_id = self.get_elite_id()
               ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_64332/2842694499.py", line 36, in get_elite_id
    self.eval()
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_64332/2842694499.py", line 49, in eval
    self.score[i] = np.sum(self.MDG(self.P[i]))
                           ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_64332/2842694499.py", line 94, in MDG
    Cov = self.phi(Cov)
          ^^^

In [49]:
#test graph
n = 10
m = 13
G = [[] for i in range(n)]
G[0] = [1]
G[1] = [0, 2, 3, 4, 5]
G[2] = [1, 5]
G[3] = [1, 4]
G[4] = [1, 3, 5, 6, 7]
G[5] = [1, 2, 4, 8]
G[6] = [4]
G[7] = [4, 8]
G[8] = [5, 7, 9]
G[9] = [8]

brkga = BRKGA(G, n, m)
S = brkga.MDG()
id = np.where(S == 1)[0]

1
0
2
3
5
4
6
7
8
9
[0 1 2 3 4 5 6 7 8 9]


In [181]:
#read file and create graph
def read_file(filename):
    inp = open(filename, 'r')
    x = inp.readlines()
    m, n, _ = map(int, x[1].strip('%').split())
    G = [[] for i in range(n)]
    for i in range(2, len(x)):
        u, v = map(int, x[i].split())
        G[u-1].append(v-1)
        G[v-1].append(u-1)
    return G, n, m

G, n, m = read_file('socfb-nips-ego/socfb-nips-ego.edges')
brkga = BRKGA(G, n, m)
S = brkga.MDG()
# print index where S[i] = 1
id = np.where(S == 1)[0]
print(len(id),id)

#brkga.run()
fast_brkga = fastBRKGA(G, n, m)
fast_brkga.run()

2888 2981
10 [   0  287  602  709  713 1524 2231 2535 2686 2698]
10.0 Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/vuhoangnguyen/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_18246/3202404575.py", line 23, in <module>
    fast_brkga.run()
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_18246/2421149744.py", line 21, in run
    elite_id = self.get_elite_id()
               ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_18246/2421149744.py", line 36, in get_elite_id
    self.eval()
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_18246/2421149744.py", line 49, in eval
    self.score[i] = np.sum(self.MDG(self.P[i]))
                           ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/lj/_xdsnnxj30zbtnxqt49qr6m00000gn/T/ipykernel_18246/2421149744.py", line 94, in MDG
    Cov = self.phi(Cov)
          ^^^

In [72]:
# ca-GrQc graph is different from ca-GrQc in the paper
def read_file(filename):
    inp = open(filename, 'r')
    x = inp.readlines()
    n, _, m = map(int, x[1].split())
    G = [[] for i in range(n)]
    for i in range(2, len(x)):
        u, v = map(int, x[i].split())
        G[u-1].append(v-1)
        G[v-1].append(u-1)
    return G, n, m

G, n, m = read_file('ca-GrQc/ca-GrQc.mtx')
brkga = BRKGA(G, n, m)
S = brkga.MDG()
# print index where S[i] = 1
id = np.where(S == 1)[0]
print(len(id),id)

4158 13422
633 [   0    3   15   21   34   40   43   44   47   54   59   65   67   73
   76   85   87   88   89   94  101  102  103  106  117  118  119  122
  135  141  150  153  155  157  168  169  171  179  182  184  190  196
  201  202  227  228  239  240  250  264  265  266  267  270  273  283
  284  286  289  296  302  306  311  316  317  323  326  332  336  338
  353  356  363  366  380  387  394  396  405  407  419  420  424  426
  428  432  436  444  453  454  462  476  483  484  486  490  497  507
  518  523  528  531  532  547  549  552  556  557  564  569  570  572
  579  588  589  590  593  594  595  596  598  600  604  605  607  616
  619  626  632  641  645  648  651  659  673  679  681  689  693  702
  705  707  716  718  738  742  746  750  762  767  775  778  789  792
  796  807  816  823  824  825  827  832  839  841  843  845  846  850
  866  873  877  884  885  894  895  896  903  906  907  912  919  925
  927  932  939  942  970  974  975  979  995  999 1003 1006 1

In [183]:
def read_file(filename):
    inp = open(filename, 'r')
    x = inp.readlines()
    n, _, m = map(int, x[1].split())
    print(n, m)
    G = [[] for i in range(n)]
    for i in range(2, len(x)):
        u, v = map(int, x[i].split())
        G[u-1].append(v-1)
        G[v-1].append(u-1)
    return G, n, m

G, n, m = read_file('socfb-Mich67/socfb-Mich67.mtx')
brkga = BRKGA(G, n, m, tlim=300)
S = brkga.MDG()
# print index where S[i] = 1
id = np.where(S == 1)[0]
print(len(id),id)

3748 81903
206 [   4   10   13   14   71   88  105  135  144  165  173  185  188  257
  269  270  276  278  302  305  315  325  339  341  406  436  439  444
  488  511  512  513  542  560  569  578  593  597  630  635  637  665
  684  724  731  751  766  813  837  891  896  915  966  967  987 1001
 1019 1021 1022 1042 1049 1056 1074 1081 1084 1086 1120 1140 1155 1185
 1194 1227 1235 1287 1298 1320 1364 1415 1464 1468 1470 1486 1488 1500
 1509 1521 1525 1542 1564 1566 1567 1571 1592 1596 1612 1629 1666 1680
 1724 1760 1813 1821 1824 1847 1862 1907 1958 1971 1995 2003 2008 2033
 2041 2124 2173 2200 2218 2228 2249 2286 2318 2322 2336 2405 2407 2415
 2429 2437 2439 2444 2473 2506 2543 2553 2558 2563 2600 2619 2639 2647
 2702 2709 2713 2720 2740 2744 2750 2754 2765 2790 2803 2807 2808 2826
 2844 2876 2906 2943 2955 2959 2972 3006 3020 3070 3075 3093 3098 3101
 3138 3145 3153 3184 3195 3199 3225 3228 3235 3244 3260 3317 3340 3350
 3360 3377 3383 3387 3427 3451 3467 3511 3515 3522 3535 3563 3

In [192]:
x = open('facebook_combined.txt', 'r').readlines()
m = len(x)
n = 4039
G = [[] for i in range(n)]
for i in range(m):
    u, v = map(int, x[i].split())
    G[u].append(v)
    G[v].append(u)

print(n, m)
brkga = BRKGA(G, n, m)
S = brkga.MDG()
# print index where S[i] = 1
id = np.where(S == 1)[0]
print(len(id),id)

4039 88234
530 [   0    4    8    9   21   25   26   41   53   56   67   78   80   89
  107  115  119  122  136  175  181  184  198  200  203  227  239  242
  249  252  271  275  277  312  320  322  326  343  346  348  353  363
  364  366  373  376  395  400  412  414  428  465  475  483  484  497
  500  513  517  538  553  559  561  563  566  573  578  579  580  583
  592  600  606  615  627  630  637  643  654  658  659  686  694  697
  698  705  713  719  724  729  745  747  776  781  804  805  818  823
  824  828  830  870  885  896  916  917  925  946  993 1006 1014 1017
 1032 1059 1067 1070 1076 1078 1085 1086 1097 1104 1118 1126 1132 1136
 1146 1177 1184 1185 1192 1197 1199 1204 1211 1216 1222 1227 1235 1238
 1277 1322 1331 1338 1345 1352 1358 1360 1367 1371 1376 1377 1390 1391
 1399 1420 1431 1459 1462 1471 1472 1479 1516 1522 1533 1536 1548 1551
 1554 1555 1557 1559 1573 1577 1583 1584 1589 1591 1596 1603 1604 1610
 1612 1613 1621 1622 1628 1636 1661 1662 1663 1684 1687 1703 1